In [7]:
import pandas as pd

In [13]:
evaluation_questions = dict()

with open("dataset/ques_ans.txt", "r") as file:
    lines = file.readlines()
    question_type = "Stock Performance"
    for line in lines:
        processed_line = line.strip()
        if question_type not in evaluation_questions:
            evaluation_questions[question_type] = []
        if processed_line != "":
            if processed_line.startswith("Q:"):
                ques_ans = dict()
                ques_ans["query"] = processed_line.removeprefix("Q: ")
            elif processed_line.startswith("A:"):
                ques_ans["response"] = processed_line.removeprefix("A: ")
                evaluation_questions[question_type].append(ques_ans)
            else:
                question_type = processed_line
    

In [14]:
import json
json.dump(evaluation_questions, open("dataset/ques_ans.json", 'w'))

In [5]:
import json
json_file = open("tests/dataset/ques_ans.json")
evaluation_queries = json.load(json_file)

In [8]:
eval_df = pd.DataFrame({
    'Category': [category for category in evaluation_queries for _ in evaluation_queries[category]],
    'Query': [query['query'] for category in evaluation_queries.values() for query in category],
    'Expectation': [query['response'] for category in evaluation_queries.values() for query in category]
})

In [9]:
import dspy
from pydantic import BaseModel, Field

class Score(BaseModel):
    commentary: str = Field(desc="The analysis of the score")
    Score: int = Field(desc="The score")

class Scorer(dspy.Signature):
    """
    You are a code evaluating agent. You take a query and code for evaluation. 
    You need to +1 for each of these attributes in the code

    {'correct_column_names','title','Annotations', 'Aggregation used',
    'correct axis label','Plotly_white theme','Correct chart type'}

    You are provided with a {query}
    and Plotly code {code}
    You need to tell me the total score
    
    """
    query = dspy.InputField(desc="user query which includes information about data and chart they want to plot")
    code = dspy.InputField(desc="The agent generated code")
    output: Score = dspy.OutputField(desc='The score after evaluating the code')


def check_code_run(code):
    score = 0
    try:
        code = code.split('```')[1]
        exec(code)
        score += 10
        return score

    except:
        return score
    
import re

def extract_integers(text):
    return [int(num) for num in re.findall(r'\d+', text)]

def evaluating_response(code, query):
    score = 0
    scorer = dspy.Predict(Scorer)
# Feeds the code and query to the LLM for evaluation
    response = scorer(query=query, code=code)
# Parses the LLM output, your implementation may vary
    print(response)
    score+=extract_integers(response.output)[0]
    return score

In [2]:
import dspy
from pydantic import BaseModel, Field
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.llm.agent_llms import get_llm_backend
from agentlite_finance.memory.shared_memory import SharedMemory
from agentlite_finance.actions.code_generation_action import CodeGenerationAction
from agentlite_finance.actions.file_handler_action import FileHandlerAction
from agentlite_finance.actions.generic_insights_action import GenericInsightsAction
from agentlite_finance.actions.plotting_action import PlottingAction

llm_config_dict =  {
                        "llm_name": "gpt-3.5-turbo",
                        "temperature": 0.0,
                        "max_tokens": 2000,
                        "api_key": "sk-Vo7jCT5lrwMyJ1YeqpzmYvDaS9sYF4Xt_BPLSaiOywT3BlbkFJVP2JXFoP36HSMWqWluMD88AkB7t0KHJ8j-FM0BUngA"
                    }
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

shared_mem = SharedMemory()
codegeneration_action = CodeGenerationAction(shared_mem)
from agentlite.logging.terminal_logger import AgentLogger
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=True) 

plotting_action = PlottingAction(shared_mem)
from agentlite_finance.agents.visualization_agent import VisualizationAgent
visualization_agent = VisualizationAgent(
    llm=llm,
    actions=[codegeneration_action, plotting_action],
    shared_mem=shared_mem,
    logger=agent_logger
)

class Plotly_code(BaseModel):
    commentary: str = Field(desc="The comments about the code")
    Code: str = Field(desc="The Plotly Code")

class AgentSig(dspy.Signature):
    """
    You are AI agent who uses the {query} to generate data visualizations in Plotly.
    You must give an output as code, in case there is no relevant columns, just state
    that you don't have the relevant information
    """
    query = dspy.InputField(desc="user query which includes information about data and chart they want to plot")
    code: Plotly_code = dspy.OutputField(desc="Plotly code that visualizes what the user needs according to the query & dataframe_index & styling_context")


class AI_data_viz_agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.agent = codegeneration_action
    
    def forward(self, query):
        prediction = self.agent(query=query)
        print(prediction)
        return dspy.Prediction(code=prediction.get('response'))


lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key="sk-Vo7jCT5lrwMyJ1YeqpzmYvDaS9sYF4Xt_BPLSaiOywT3BlbkFJVP2JXFoP36HSMWqWluMD88AkB7t0KHJ8j-FM0BUngA")
dspy.configure(lm=lm)
agent=AI_data_viz_agent()
print(agent('What is the moving average of AAL stock?').code)

#############human_template: {prompt}
#############PromptTemplate(prompt): input_variables=['prompt'] template='{prompt}'
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by other entity. Include all the 
                                                required imports in the code. Take help of data sample and su

2024-09-30 19:05:19.780 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:05:19.851 
  command:

    streamlit run /Users/kamalkis/Documents/Personal/Capstone/poc_cap/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-30 19:05:19.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:05:19.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:05:19.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    df['Moving Average'] = df['Close'].rolling(window=20).mean()\n\n    fig = px.line(df, x=df.index, y=['Close', 'Moving Average'], title='Stock Data with Moving Average')\n    \n    st.plotly_chart(fig)\n\n# Example of how to use the method (don't call it here)\n# df = pd.read_csv('stock_data.csv')\n# plot_chart_for_stock_data(df)"}
import pandas as pd
import plotly.express as px
import streamlit as st

def plot_chart_for_stock_data(df):
    df['Moving Average'] = df['Close'].rolling(window=20).mean()

    fig = px.line(df, x=df.index, y=['Close', 'Moving Average'], title='Stock Data with Moving Average')
    
    st.plotly_chart(fig)

# Example of how to use the method (don't call it here)
# df = pd.read_csv('stock_data.csv')
# plot_chart_for_stock_data(df)


In [10]:
code_list =[]
for q in eval_df['Query']:
    code_list.append(agent(q).code)
eval_df['Code'] = code_list

eval_df['check_run'] = [check_code_run(code) for code in eval_df['Code']]

eval_df['Attribute_Score'] = [evaluating_response(code,query) for code,query in zip(eval_df['Code'],eval_df['Query'])]

eval_df['Answerable'] = [1 if x.strip().lower()!='no relevant information' else 0 for x in eval_df['Expectation']]


***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 
                                                method, It will be invoked by other entity. Include all the 
                                                required imports in the code. Take help of data sample and summary
                                                to generate your code. Do not initialize dataframe on your own.
    

2024-09-30 19:07:01.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:01.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:01.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:01.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(dataframe):\n    fig = px.line(dataframe, x='Date', y='Close', title='Stock Closing Prices')\n    st.plotly_chart(fig)\n\n# Sample usage:\n# Assuming you have a pandas dataframe named 'stock_data' containing stock data with columns 'Date' and 'Close'\n# plot_chart_for_stock_data(stock_data)"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                   

2024-09-30 19:07:02.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:02.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:02.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:07:02.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Close', title='Stock Closing Prices')\n    st.plotly_chart(fig)\n\n# Example usage:\n# Assuming df is the pandas dataframe containing stock data\n# plot_chart_for_stock_data(df)"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig). Do not call the 

...

{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    df['Date'] = pd.to_datetime(df['Date'])\n    \n    fig = px.line(df, x='Date', y='Close', title='Stock Price Trend for Microsoft')\n    \n    st.plotly_chart(fig)\n\n# Example usage:\n# plot_chart_for_stock_data(your_dataframe)"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like st.plotly_chart(fig

2024-09-30 19:11:30.613 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:30.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:30.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:30.618 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    fig = px.line(df, x='Date', y='Stock Price', title='Stock Price Trend')\n    st.plotly_chart(fig)\n\n# This method will plot the stock price trend using the provided dataframe\n# Make sure to pass a dataframe with columns 'Date' and 'Stock Price'"}
***** CODEGEN LLM PROMPTInstructions: 
 Give me a python method with the name plot_chart_for_stock_data which takes
                                                pandas dataframe as argument. Use your knowledge to generate
                                                accurate python code of the charts using plotly for the user
                                                prompt given below. Do not include the method in any class. 
                                                And this method should print the plot in streamlit UI using
                                                plotly_chart API like 

2024-09-30 19:11:32.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:32.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:32.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 19:11:32.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'response': "import pandas as pd\nimport plotly.express as px\nimport streamlit as st\n\ndef plot_chart_for_stock_data(df):\n    df['Date'] = pd.to_datetime(df['Date'])\n    df = df.set_index('Date')\n    \n    fig = px.line(df, x=df.index, y='Close', title='Stock Price Movement')\n    st.plotly_chart(fig)\n\n# Example usage:\n# plot_chart_for_stock_data(your_dataframe)"}
2024-09-30 19:11:32 - 	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated. ***
 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
2024-09-30 19:11:33 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Prediction(
    output='Output: 4'
)
2024-09-30 19:11:34 - HTTP Requ

In [11]:
def final_score(a,c,a_s,code):
    return a * (c + a_s)

In [12]:
class CodeJudge(dspy.Signature):
    """Judge if the response has any code"""
    response = dspy.InputField(desc="Response from AI agent")
    has_code = dspy.OutputField(desc="Does the response contain any Python code", prefix="Factual[Yes/No]:")

# A metric that calculates the final score for every predicted response
# compared with the example which contains the best response
def full_metric(example,pred, trace=None):
    if 'No relevant information' not in example.code:
        check_run = check_code_run(pred.code)
        attributes = evaluating_response(pred.code,example.query)
    else:
        check_if_code = dspy.ChainOfThought(CodeJudge)
        response = check_if_code(response=pred.code)
        if response.has_code.split('Factual[Yes/No]:')[1].strip()=='Yes':
            return 0
        else:
            return 19
        
    return check_run + attributes

zip_ = zip(eval_df['Answerable'],eval_df['check_run'],eval_df['Attribute_Score'],eval_df['Code'])
eval_df['Total_Score'] = [final_score(a,c,a_s,code) for a,c,a_s,code in zip_]

# computing the total score/total attainable score

eval_df['Total_Score'].sum()/(len(eval_df)*19)

0.26894736842105266

In [13]:
print("Intial score: ",format(eval_df['Total_Score'].sum()/(len(eval_df)*19)*100, '0.2f') + "%")

Intial score:  26.89%


In [16]:
eval_df.sample(10)

,Category,Query,Expectation,Code,check_run,Attribute_Score,Answerable,Total_Score
64,Event-Driven,How did Facebook’s stock respond after the ear...,Facebook’s stock rose by 4.1% the following day.,import pandas as pd\nimport plotly.express as ...,0,7,1,7
71,Event-Driven,What happened to Amazon’s stock after the earn...,Amazon’s stock rose by 6.1% the following day.,import pandas as pd\nimport plotly.express as ...,0,4,1,4
1,Stock Performance,What was the closing price of Apple on Februar...,$157.16,import pandas as pd\nimport plotly.express as ...,0,7,1,7
24,Stock Performance,How much did Microsoft’s stock rise between Ap...,10.2%,import pandas as pd\nimport plotly.express as ...,0,7,1,7
83,Time-Series,How did Amazon’s stock perform between January...,"Amazon’s average stock price was $3,127.80, wi...",import pandas as pd\nimport plotly.express as ...,0,4,1,4
96,Time-Series,How much did Google’s stock increase between M...,Google’s stock increased by 4.5%.,import pandas as pd\nimport plotly.express as ...,0,7,1,7
13,Stock Performance,What was the closing price of Google on July 1...,"$2,743.90",import pandas as pd\nimport plotly.express as ...,0,7,1,7
88,Time-Series,What was the lowest stock price for Netflix be...,The lowest price was $351.21.,import pandas as pd\nimport plotly.express as ...,0,7,1,7
87,Time-Series,What was Facebook’s highest stock price betwee...,The highest price was $279.48.,import pandas as pd\nimport plotly.express as ...,0,5,1,5
49,Comparative Performance,How did Tesla’s stock price compare to Microso...,"Tesla underperformed Microsoft, with a 4% decr...",import pandas as pd\nimport plotly.express as ...,0,5,1,5
